# crawl all den haag woo dossiers

* marx
* 2022-12-20, updated 2023-09-26

## Sjaboon

[Den Haag s Woo dossiers](https://woo.denhaag.nl/wob-besluiten/) zijn een mooi voorbeeld van een publicatie-methode die heel goed netjes te scrape  valt. Het bestaat uit 3 onderdelen:

1. Een pagina met een lijst van alle woo-dossiers,  met een link naar een aparte pagina voor elk dossier.
    * Het gebruik van "pagination in de URL" om naar de volgende pagina in die lijst te gaan.
    * **Stap 1 i het extractiescript gebruikt dit.**
2. Elk Woo-dossier heeft een eigen vaste pagina met een eigen URL.
    * Die worden in **stap 2** verzameld inn een lijst.
3. Elke pagina met een Woo-dossier heeft een vaste structuur die via XPad om te zettenn is naar ons schema.
    * Meta data op de HTML pagina met een vast schema (dc, og  of iets vergelijkbaars) maakt het verzamelen van metadata op dossier  niveau vaak eenvoudiger en robuuster.
    * De extractioe per dossier gebeurt in **stap 3**.
4. Tenslotte doenn we de extractie over alle urls van xde woo-dossiers, en slaan dat allemaal op in eeen json datafile. **stap 4 en 5**.

In [1]:

import re
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import json
import requests
import gzip

# globals

In [2]:
DC_TYPE = '2i'
DC_PUBLISHER='gm0518'
DC_CREATOR='Gemeente Den Haag'

# Extractie functies

In [3]:
def crawl_all_pages(url,page_range=range(100)):
    '''Crawl all pages from a paginated list page with URL and pagerange.
    Simply concatenate all of them and return that as a strng. 
    From that string we will extract all pages we want'''
    pages=''
    for p in page_range:
        u= url+str(p)
        print (u)
        ef = !curl -s $u
        pages=pages+'\n'+ ''.join(ef)
    return pages




def xpadje(element,path_expression):
    if element.xpath(path_expression):
        return element.xpath(path_expression)[0]
    else:
        
        return ''
    
def pak_meta(element,property_value='dc:title'):
    '''Geef de waarde van het @content attribuut in het meta element met @property waarde property_value.
    Als het niet bestaat geef lege string'''
    xpad= './/meta[@property="WAARDE"]/@content'.replace('WAARDE',property_value)
    return xpadje(element, xpad)
    
  
    
def extract_links(woodata,dc_source_xpad='@href', dc_title_xpad='.//text()'):
    '''Extract the list of document dicts from woodata which is a list of a elements from the HTML page.
    One usually only has to finetune the dc_title_xpad'''
    L=[]
    for e in woodata:
        D=dict()
        
        D["dc_source"]= xpadje(e,dc_source_xpad)
        D["dc_title"]= ''.join(e.xpath(dc_title_xpad)).strip() 
        if 'besluit'.lower() in D["dc_title"].lower() :
            D["dc_type"]='besluit' 
        elif 'verzoek'.lower() in D["dc_title"].lower() :
            D["dc_type"]='verzoek' 
        elif 'inventaris'.lower() in D["dc_title"].lower() :
            D["dc_type"]='inventarislijst' 
        else:
            D["dc_type"]='bijlage' 
        D["foi_url_on_web"]= True #'true'
        L.append(D) 
    return  L

# step 1 crawl all pages with links to woo dossiers

In [4]:
URL='https://woo.denhaag.nl/wob-besluiten/?_paged='
page_range=range(1,80) ## todo dit moet nog uitgeprogrameeerd worden
pages = crawl_all_pages(URL,page_range )

https://woo.denhaag.nl/wob-besluiten/?_paged=1
https://woo.denhaag.nl/wob-besluiten/?_paged=2
https://woo.denhaag.nl/wob-besluiten/?_paged=3
https://woo.denhaag.nl/wob-besluiten/?_paged=4
https://woo.denhaag.nl/wob-besluiten/?_paged=5
https://woo.denhaag.nl/wob-besluiten/?_paged=6
https://woo.denhaag.nl/wob-besluiten/?_paged=7
https://woo.denhaag.nl/wob-besluiten/?_paged=8
https://woo.denhaag.nl/wob-besluiten/?_paged=9
https://woo.denhaag.nl/wob-besluiten/?_paged=10
https://woo.denhaag.nl/wob-besluiten/?_paged=11
https://woo.denhaag.nl/wob-besluiten/?_paged=12
https://woo.denhaag.nl/wob-besluiten/?_paged=13
https://woo.denhaag.nl/wob-besluiten/?_paged=14
https://woo.denhaag.nl/wob-besluiten/?_paged=15
https://woo.denhaag.nl/wob-besluiten/?_paged=16
https://woo.denhaag.nl/wob-besluiten/?_paged=17
https://woo.denhaag.nl/wob-besluiten/?_paged=18
https://woo.denhaag.nl/wob-besluiten/?_paged=19
https://woo.denhaag.nl/wob-besluiten/?_paged=20
https://woo.denhaag.nl/wob-besluiten/?_paged=21
h

# step 2 extract links to woo dosiers

* fine tune 

In [5]:
woos=re.findall(r'href="(.*?)"',pages)
woos= [u for u in woos if u.startswith('https://woo.denhaag.nl/wob-verzoek')  ]
len(woos), len(set(woos)), woos[:5]

(531,
 531,
 ['https://woo.denhaag.nl/wob-verzoek/2023-230426-woo-inzake-apelloods-op-het-adres-laan-van-poot-97/',
  'https://woo.denhaag.nl/wob-verzoek/2023-224378-woo-project-nieuw-waldeck-knapt-op/',
  'https://woo.denhaag.nl/wob-verzoek/2023-235904-woo-overzicht-ingediende-lopende-woo-verzoeken-en-uitgekeerde-dwangsommen/',
  'https://woo.denhaag.nl/wob-verzoek/2023-229668-woo-inzake-appelloods-op-het-adres-laan-van-poot-97/',
  'https://woo.denhaag.nl/wob-verzoek/2023-219282-woo-inzake-apeldoornselaan-266/'])

# step 3 extract info for each woo dossier

## kan ook uit metada

```
 <!-- This site is optimized with the Yoast SEO plugin v20.6 - https://yoast.com/wordpress/plugins/seo/ -->
	<title>165143-WOO-inzake de communicatie tussen burgemeester Van Zanen en Shell - WOO-verzoeken Den Haag</title>
	<meta property="og:locale" content="nl_NL" />
	<meta property="og:type" content="article" />
	<meta property="og:title" content="165143-WOO-inzake de communicatie tussen burgemeester Van Zanen en Shell - WOO-verzoeken Den Haag" />
	<meta property="og:url" content="https://woo.denhaag.nl/wob-verzoek/165143-woo-inzake-de-communicatie-tussen-burgemeester-van-zanen-en-shell/" />
	<meta property="og:site_name" content="WOO-verzoeken Den Haag" />
	<meta property="article:modified_time" content="2022-12-12T13:11:56+00:00" />
	<meta property="og:image" content="https://woo.denhaag.nl/wp-content//uploads/2021/06/Share-image-scaled.jpg" />
	<meta property="og:image:width" content="1920" />
	<meta property="og:image:height" content="1008" />
	<meta property="og:image:type" content="image/jpeg" />
	<meta name="twitter:card" content="summary_large_image" />
	 


```
 

In [6]:
## Dit kan ook heel makkelijk uit de metadata 

def parse_one_woo(url):
    #print(url)
    # get the page of url as an etree object dom 
    ef= !curl $url
    soup=' '.join(ef)
    #print(soup.prettify())
    #with open(p) as fp:
    #    soup = BeautifulSoup(fp)
    dom = etree.HTML(str(soup))
    # now extract in ww
    woo=dict()
    woo['dc_source']=pak_meta(dom,'og:url')
    woo['dc_title']=pak_meta(dom,'og:title')
    mydate= ''.join(dom.xpath('//p[@class="date"]//text()')).strip() #re.sub(r'T.*','',pak_meta(dom,'article:modified_time'))
    woo['foi_publishedDate']='-'.join(mydate.split('-')[::-1])
    #woo['dc_description']= pak_meta(dom,'og:description')
    
    woo['dc_creator']=DC_CREATOR
    woo["dc_type"]= DC_TYPE
    woo["dc_publisher"]=  DC_PUBLISHER
    woodata= dom.xpath('//li[@class="link"]//a')
    
    if woodata:
        woo['foi_files']={"foi_documenten": 
                          extract_links(woodata,dc_title_xpad='./span[@class="button-title"]/text()')
                         }
         
    return woo

# test
#parse_one_woo('https://woo.denhaag.nl/wob-verzoek/165143-woo-inzake-de-communicatie-tussen-burgemeester-van-zanen-en-shell/')


# stap 4

* Extraheer u de data voor elk woodossier i n `woos`.

In [7]:
%%time
D=list()
for url in woos:
    try:
        o= parse_one_woo(url)
        D.append(o)
    except:
        print('WRONG',url)

len(D)

CPU times: user 2.31 s, sys: 3.73 s, total: 6.04 s
Wall time: 30min 37s


531

# step 5 store as json

In [8]:
jsonfilename=DC_PUBLISHER+'json.gz'
with gzip.open(jsonfilename, 'w') as fout:
    fout.write(json.dumps(D).encode('utf-8'))  
!ls -lh $jsonfilename

-rw-r--r--  1 admin  staff    56K Sep 27 16:50 gm0518json.gz


In [9]:
# if you want to open the file again
with gzip.open(jsonfilename, 'r') as fin:
    data = json.loads(fin.read().decode('utf-8'))
len(data),data[:2]

(531,
 [{'dc_source': 'https://woo.denhaag.nl/wob-verzoek/2023-230426-woo-inzake-apelloods-op-het-adres-laan-van-poot-97/',
   'dc_title': '2023-230426-WOO-inzake Apelloods op het adres Laan van Poot 97 - WOO-verzoeken Den Haag',
   'foi_publishedDate': '2023-09-13',
   'dc_creator': 'Gemeente Den Haag',
   'dc_type': '2i',
   'dc_publisher': 'gm0518',
   'foi_files': {'foi_documenten': [{'dc_source': 'https://woo.denhaag.nl/wob-verzoek/2023-230426-woo-inzake-apelloods-op-het-adres-laan-van-poot-97/2023-230426-woo-verzoek/',
      'dc_title': '2023-230426 Woo-verzoek',
      'dc_type': 'verzoek',
      'foi_url_on_web': True},
     {'dc_source': 'https://woo.denhaag.nl/wob-verzoek/2023-230426-woo-inzake-apelloods-op-het-adres-laan-van-poot-97/2023-230426-woo-besluit/',
      'dc_title': '2023-230426 Woo-besluit',
      'dc_type': 'besluit',
      'foi_url_on_web': True},
     {'dc_source': 'https://woo.denhaag.nl/wob-verzoek/2023-230426-woo-inzake-apelloods-op-het-adres-laan-van-poot-9